# Minimum Edit Distance and Dynamic Programming

A common task in NLP is to measure how similar two different texts are. An old school way of approaching this problem, sans neural networks, is to simply measure how many additions and deletions would it take to start from text A and arrive at text B. For example we could imagine news headlines which are in down to the character level very similar:

    "President Obama spoke today in the Rose Garden"
    "Barack Obama spoke in the Rose Garden"
    
Strictly speaking the strings are different, however they are very similar in wording, and even convey similar meaning.

The challenge with determining how many edits does it take to start from text A and arrive at text B is that we must **search** for the the fewest amount of edits possible. To achieve this we will rely on **dynamic programming** which is a class of algorithms which relies on tables to solve the problem. Two examples which will come up frequently are **Viterbi** and **CKY**.

The key idea in such algorithms is that we incrementally brute force all possibilities, however we keep track of each step and its associated cost. Thus when we have completed trying all possible combinations, we can retrace the steps which led to the lowest cost solution. To illustrate this concept we can program the algorithm. After that we will quickly review a few libraries which help you do this.

### Minimum Edit Distance Implementation

As as simple example, assume we want to find how few changes does it require to get from the single word 'intention' to the 'execution'. How many changes does it take?

In [2]:
import itertools

In [25]:
import numpy as np
import pandas as pd

def substitution_logic(source_letter, target_letter, cost):
    if source_letter == target_letter:
        return 0
    else:
        return cost

def min_edit_distance(source, target, insert_cost=1, delete_cost=1, substitute_cost=2):
    source = '#{}'.format(source)
    target = '#{}'.format(target)
    n = len(source)
    m = len(target)

    table = np.zeros((n,m))

    for row in range(1,n):
        table[row,0] = table[row-1,0] + delete_cost
    for col in range(1,m):
        table[0, col] = table[0, col-1] + insert_cost

    for row, col in itertools.product(range(1,n), range(1,m)):
        table[row, col] = min(table[row-1,col] + delete_cost, 
                              table[row-1,col-1] + substitution_logic(source[row], target[col], substitute_cost), 
                              table[row, col-1] + insert_cost)
    df = pd.DataFrame(res, columns=list(target), dtype=int)
    df.index = list(source)
    return df

In [26]:
source = 'intention'
target = 'execution'

min_edit_distance(source, target)

,#,e,x,e,c,u,t,i,o,n
#,0,1,2,3,4,5,6,7,8,9
i,1,2,3,4,5,6,7,6,7,8
n,2,3,4,5,6,7,8,7,8,7
t,3,4,5,6,7,8,7,8,9,8
e,4,3,4,5,6,7,8,9,10,9
n,5,4,5,6,7,8,9,10,11,10
t,6,5,6,7,8,9,8,9,10,11
i,7,6,7,8,9,10,9,8,9,10
o,8,7,8,9,10,11,10,9,8,9
n,9,8,9,10,11,12,11,10,9,8


Having generated the table of results we can trace the path back from the lower right diagnol to the upper left corner traversing either left, up, or diagnol which indicate an insertion, substitution, and deletion respectively.

It should be noted that the costs defined as the default values in the above function are referred to as the Levenshtein distance. In this metric, substitutions are equivalent to one deletion and one addition. Certainly one could adjust these costs if it makes sense for the given use case.

In the next section let's take a look at a few libaries which offer this functionality in prebuilt python packages

## `textdistance` and `fuzzywuzzy`

Two popular python libraries for accomplish this task are [`textdistance`](https://github.com/life4/textdistance) and [`fuzzywuzzy`](https://github.com/seatgeek/fuzzywuzzy).  In addition the `textdistance` repo links to a few nice resources such as [this one](http://theautomatic.net/2019/11/13/guide-to-fuzzy-matching-with-python/) about use cases for such libraries and the various algorithms.

In [30]:
import textdistance
from fuzzywuzzy import fuzz

In [28]:
source = 'Obama was seen talking to reporters on Tuesday'
target = 'President Obama spoke to the press Tuesday afternoon'

In [36]:
%%time
textdistance.levenshtein.distance(source, target)

CPU times: user 30 µs, sys: 1e+03 ns, total: 31 µs
Wall time: 34.8 µs


39

In [33]:
%%time
fuzz.ratio(source, target)

CPU times: user 20 µs, sys: 0 ns, total: 20 µs
Wall time: 21.7 µs


51

As is seen, the results that `fuzzywuzzy` and `textdistance` deliver are totally different. `fuzzywuzzy` focuses on simplicity, supporting only levenshtein distance and some convenience functions to process a string against multiple targets to find the most similar. In contrast `textdistance` offers a wider variety of algorithms which extend beyond character based edit distance/similarity. For more information certainly check out the repositories linked above.